<a href="https://colab.research.google.com/github/chanseongparkk/BigDataAnalyst_practise/blob/main/HR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd drive/My Drive/빅데이터분석기사 실기/HR

/content/drive/My Drive/빅데이터분석기사 실기/HR


In [ ]:
import pandas as pd
import numpy as np

X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

X_train_id = X_train.iloc[:,0]
X_test_id = X_test.iloc[:,0]
y_train_id = y_train.iloc[:,0]

X_train = X_train.iloc[:,1:]
X_test = X_test.iloc[:,1:]
y_train = y_train.iloc[:,1:]
#print(X_train.info(),X_test.info(),y_train.info())

### 결측치

#print(X_train.isna().sum(), X_test.isna().sum(),X_train.shape,X_test.shape)

# 결측치 많은 변수 날리기, 개체의 수가 많은 변수 날리기

X_train = X_train.iloc[:,[1,3,4,5,7,10,11]]
X_test = X_test.iloc[:,[1,3,4,5,7,10,11]]

# print(X_train.shape,X_test.shape)

### 결측치 대체

#print(X_train.info(),X_test.info(),y_train.info())
#print(X_train.describe(include = "object"),X_test.describe(include = "object"))
X_train.enrolled_university = X_train.enrolled_university.fillna("no_enrollment")
X_train.education_level = X_train.education_level.fillna("Graduate")
X_train.experience = X_train.experience.fillna(">20")
X_train.last_new_job = X_train.last_new_job.fillna("1")
X_test.enrolled_university = X_test.enrolled_university.fillna("no_enrollment")
X_test.education_level = X_test.education_level.fillna("Graduate")
X_test.experience = X_test.experience.fillna(">20")
X_test.last_new_job = X_test.last_new_job.fillna("1")

### 변수 변환
#print(X_train.info())
#print(X_train.skew(),X_train.kurt(),
#      np.log1p(X_train.iloc[:,[0,6]]).skew(),np.log1p(X_train.iloc[:,[0,6]]).kurt(),
#      np.sqrt(X_train.iloc[:,[0,6]]).skew(),np.sqrt(X_train.iloc[:,[0,6]]).kurt())

X_train.iloc[:,6] = np.log1p(X_train.iloc[:,6])
X_test.iloc[:,6] = np.log1p(X_test.iloc[:,6])

### 스케일러

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train.iloc[:,[0,6]] = scaler.fit_transform(X_train.iloc[:,[0,6]])
X_test.iloc[:,[0,6]] = scaler.transform(X_test.iloc[:,[0,6]])

#display(X_train.describe(include = "object"),X_test.describe(include = "object"))

### 더미변수

X_train = pd.get_dummies(X_train,drop_first = True)
X_test = pd.get_dummies(X_test,drop_first = True)
#print(X_train.shape,X_test.shape)
#print(set(X_train.columns)-set(X_test.columns)) #변수 갯수 동일

### 특수문자 들어간 변수 이름 바꾸기

#print(X_train.columns)
X_train.columns = ['city_development_index', 'training_hours',
       'relevent_experience_No relevent experience',
       'enrolled_university_Part time course',
       'enrolled_university_no_enrollment', 'education_level_High School',
       'education_level_Masters', 'education_level_Phd',
       'education_level_Primary School', 'experience_10', 'experience_11',
       'experience_12', 'experience_13', 'experience_14', 'experience_15',
       'experience_16', 'experience_17', 'experience_18', 'experience_19',
       'experience_2', 'experience_20', 'experience_3', 'experience_4',
       'experience_5', 'experience_6', 'experience_7', 'experience_8',
       'experience_9', 'experience_under1', 'experience_upper20', 'last_new_job_2',
       'last_new_job_3', 'last_new_job_4', 'last_new_job_upper4',
       'last_new_job_never']

X_test.columns = ['city_development_index', 'training_hours',
       'relevent_experience_No relevent experience',
       'enrolled_university_Part time course',
       'enrolled_university_no_enrollment', 'education_level_High School',
       'education_level_Masters', 'education_level_Phd',
       'education_level_Primary School', 'experience_10', 'experience_11',
       'experience_12', 'experience_13', 'experience_14', 'experience_15',
       'experience_16', 'experience_17', 'experience_18', 'experience_19',
       'experience_2', 'experience_20', 'experience_3', 'experience_4',
       'experience_5', 'experience_6', 'experience_7', 'experience_8',
       'experience_9', 'experience_under1', 'experience_upper20', 'last_new_job_2',
       'last_new_job_3', 'last_new_job_4', 'last_new_job_upper4',
       'last_new_job_never']

### 학습, 검증 데이터 분할

from sklearn.model_selection import train_test_split

X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state = 999)
#print(X_train2.shape, X_val.shape, y_train2.shape, y_val.shape)


### 모델링(RandomForestClassifier)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

#rf = RandomForestClassifier(random_state = 999 )
rf = RandomForestClassifier(n_estimators = 500, max_depth = 10, min_samples_split = 5, random_state = 999 )
rf.fit(X_train2,y_train2)

### 모델링(XGBClassifier)

from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators = 500, max_depth = 10, subsample = 0.3, learning_rate = 0.05, random_state = 999)
#xgb = XGBClassifier(random_state = 999)
xgb.fit(X_train2,y_train2,eval_metric = "auc")

print("rf train accuracy :", rf.score(X_train2,y_train2),"\n",
      "rf train roc_auc_score :", roc_auc_score(y_train2,rf.predict(X_train2)),"\n",
      "rf validation accuracy :", rf.score(X_val,y_val),"\n",
      "rf validation roc_auc_score :", roc_auc_score(y_val,rf.predict(X_val)),"\n",
      "xgb train accuracy :", xgb.score(X_train2,y_train2),"\n",
      "xgb train roc_auc_score :", roc_auc_score(y_train2,xgb.predict(X_train2)),"\n",
      "xgb validation accuracy :", xgb.score(X_val,y_val),"\n",
      "xgb validation roc_auc_score :", roc_auc_score(y_val,xgb.predict(X_val)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


rf train accuracy : 0.8041014168530947 
 rf train roc_auc_score : 0.6765769192801346 
 rf validation accuracy : 0.7755741127348643 
 rf validation roc_auc_score : 0.6291050117509479 
 xgb train accuracy : 0.8771812080536913 
 xgb train roc_auc_score : 0.7919421714984218 
 xgb validation accuracy : 0.756437021572721 
 xgb validation roc_auc_score : 0.6150014112959835
